<a href="https://colab.research.google.com/github/Anali24/SIMULACION-I/blob/main/Ejemplo5_7_SI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Ejemplo 5.7 Libros Coss Bu***

Un vendedor de revistas compra mensualmente una revista el día primero de cada mes. El costo de cada ejemplar es de $1.50.La demanda de esta revista en los primeros 10 días del mes sigue la siguiente distribución de probabilidad:



| Demanda |  5   |  6   |  7   |  8   |  9   |  10  |  11  |
|---------|------|------|------|------|------|------|------|
| Prob.   | 0.05 | 0.05 | 0.10 | 0.15 | 0.25 | 0.25 | 0.15 |


Al final del décimo día, el vendedor puede regresar cualquier cantidad al proveedor, quien se las pagará a $0.90$ el ejemplar, o comprar más a $1.20$ el ejemplar. La demanda en los siguientes 20 días está dada por la siguiente distribución de probabilidad:

| Demanda |  4   |  5   |  6   |  7   |  8   |
|---------|------|------|------|------|------|
| Prob.   | 0.15 | 0.20 | 0.30 | 0.20 | 0.15 |


Al final del mes, el vendedor puede regresar al proveedor las revistas que le sobren, las cuales se le pagarán a $0.60$ el ejemplar.
Finalmente, se asume que después de un mes ya no existe demanda por parte del público, puesto que para ese entonces ya habrá aparecido el muevo numero en la revista
Si el precio al público es de $2$ por ejemplar, determine la política óptima de compra.

**Solución y Planteamiento para la simulación:**

Dados los ingresos y los costos , queremos maximizar la ganancia, haciendo correctamente el pedido inicial.

Así la ganancia esperada para la demanda 1 es:


$$ E[G(Q_1)] = \sum_{d_1} P(d_1) \cdot G(Q_1, d_1)  $$
La ganancia $ G(Q_1, d_1) $ se calcula como:

$$
G(Q_1, d_1)=
\begin{cases}
(Q_1 - d_1) \cdot 0.90 & \text{si } Q_1 > d_1 \\
(d_1 - Q_1) \cdot (-1.20) & \text{si } d_1 > Q_1
\end{cases}
$$

Ahora la Ganancia esperada en los siguientes 20 días.

Para el inventario restante después del décimo día $ I_1 = Q_1 - d_1 $:

$$
E[G_2(I_1)] = \sum_{d_2} P(d_2) \cdot G_2(I_1, d_2)
$$
donde $ d_2 $ es la demanda en los siguientes 20 días.

La ganancia $ G_2(I_1, d_2) $ se calcula como:

$$
G_2(I_1, d_2) =
\begin{cases}
(I_1 - d_2) \cdot 0.60 & \text{si } I_1 > d_2 \\
(d_2 - I_1) \cdot 2.00 - I_1 \cdot 1.50 & \text{si } d_2 > I_1
\end{cases}
$$

Optimización
Calculamos la ganancia esperada total:

$$
E[G_{\text{total}}(Q_1)] = E[G(Q_1)] + E[G_2(I_1)]
$$

Maximizamos $ E[G_{\text{total}}(Q_1)] $ respecto a $ Q_1 $.

Vamos a hacer los cálculos para cada $ Q_1 $:

Calcular $ E[G(Q_1)] $ para $ Q_1 = 5, 6, 7, 8, 9, 10, 11 $.

Calcular $ E[G_2(I_1)] $ para cada inventario restante $ I_1 $.


Usaremos **Monte Carlo** para estimar la ganancia esperada para cada cantidad inicial de revistas compradas (Q1) y determina cuál es la mejor estrategia de compra inicial basada en la ganancia esperada. La cantidad óptima de compra inicial se determina como aquella que maximiza la ganancia esperada.

Monte Carlo es ideal cuando hay elementos de incertidumbre en el proceso de toma de decisiones, como las demandas de los clientes que varían según probabilidades específicas en períodos de tiempo diferentes (primeros 10 días y siguientes 20 días en este caso).

***Simulación:***

In [57]:
import numpy as np

# Definición de las distribuciones de probabilidad
prob_demanda_10_dias = [0.05, 0.05, 0.10, 0.15, 0.25, 0.25, 0.15]
demanda_10_dias = [5, 6, 7, 8, 9, 10, 11]

prob_demanda_20_dias = [0.15, 0.20, 0.30, 0.20, 0.15]
demanda_20_dias = [4, 5, 6, 7, 8]


In [58]:
# Parametros de costos e ingresos
costo_compra_inicial = 1.50
precio_venta = 2.00
costo_devolucion_10_dias = 0.90
costo_compra_adicional_10_dias = 1.20
costo_devolucion_final = 0.60


In [59]:
# Función para calcular la ganancia esperada
def calcular_ganancia(Q1):
    ganancia_total = 0

    for _ in range(num_simulaciones):
        # Simular demanda en los primeros 10 días
        demanda_10 = np.random.choice(demanda_10_dias, p=prob_10_dias)

        # Ajustar inventario al décimo día
        if Q1 > demanda_10:
            # Devolución de exceso
            exceso = Q1 - demanda_10
            ganancia = demanda_10 * precio_venta - Q1 * costo_compra_inicial + exceso * costo_devolucion_10_dias
            inventario_restante = 0
        else:
            # Compra adicional
            deficit = demanda_10 - Q1
            ganancia = Q1 * precio_venta - Q1 * costo_compra_inicial - deficit * costo_compra_adicional_10_dias
            inventario_restante = deficit

        # Simular demanda en los siguientes 20 días
        demanda_20 = np.random.choice(demanda_20_dias, p=prob_20_dias)

        if inventario_restante > demanda_20:
            # Devolución de exceso al final del mes
            exceso_final = inventario_restante - demanda_20
            ganancia += demanda_20 * precio_venta + exceso_final * costo_devolucion_final
        else:
            ganancia += inventario_restante * precio_venta

        ganancia_total += ganancia

    ganancia_promedio = ganancia_total / num_simulaciones
    return ganancia_promedio


In [60]:
# Encontrar la cantidad óptima de compra inicial
resultados = []
for Q1 in range(5, 12):
    ganancia_esperada = calcular_ganancia(Q1)
    resultados.append((Q1, ganancia_esperada))

# Imprimir los resultados
for Q1, ganancia in resultados:
    print(f"Compra inicial: {Q1} revistas - Ganancia esperada: ${ganancia:.2f}")

# Determinar la cantidad óptima de compra
Q_optimo = max(resultados, key=lambda x: x[1])[0]
print(f"\nCantidad óptima de compra inicial: {Q_optimo} revistas")


Compra inicial: 5 revistas - Ganancia esperada: $5.42
Compra inicial: 6 revistas - Ganancia esperada: $5.25
Compra inicial: 7 revistas - Ganancia esperada: $4.95
Compra inicial: 8 revistas - Ganancia esperada: $4.58
Compra inicial: 9 revistas - Ganancia esperada: $4.17
Compra inicial: 10 revistas - Ganancia esperada: $3.69
Compra inicial: 11 revistas - Ganancia esperada: $3.14

Cantidad óptima de compra inicial: 5 revistas
